In [3]:
# Re-import necessary libraries since execution state was reset
import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# BEA API Key
API_KEY = "AF24D2AB-0062-43E0-A879-E47F0BAACE62"

# BEA API URL
BEA_URL = "https://apps.bea.gov/api/data"

# Parameters for Real GDP (RGDP) in Chained Dollars (Table 1.1.6)
rgdp_params = {
    "UserID": API_KEY,
    "method": "GetData",
    "datasetname": "NIPA",
    "TableName": "T10106",  # Table 1.1.6 for Real GDP (Chained Dollars)
    "Frequency": "A",  # Annual Data
    "Year": "ALL",  # Fetch all available years
    "ResultFormat": "json"
}

# Parameters for GDP Deflator (Table 1.1.9)
gdp_deflator_params = {
    "UserID": API_KEY,
    "method": "GetData",
    "datasetname": "NIPA",
    "TableName": "T10109",  # Table 1.1.9 for GDP Implicit Price Deflator
    "Frequency": "A",  # Annual Data
    "Year": "ALL",  # Fetch all available years
    "ResultFormat": "json"
}

# Parameters for Unemployment Rate (Table 6.0.0.1)
unemployment_params = {
    "UserID": API_KEY,
    "method": "GetData",
    "datasetname": "NIPA",
    "TableName": "T600001",  # Table for Unemployment Rate
    "Frequency": "A",  # Annual Data
    "Year": "ALL",  # Fetch all available years
    "ResultFormat": "json"
}

# Fetch Data from BEA API
rgdp_response = requests.get(BEA_URL, params=rgdp_params)
gdp_deflator_response = requests.get(BEA_URL, params=gdp_deflator_params)
unemployment_response = requests.get(BEA_URL, params=unemployment_params)

rgdp_data = rgdp_response.json()
gdp_deflator_data = gdp_deflator_response.json()
unemployment_data = unemployment_response.json()

# Extract RGDP Data
rgdp_records = []
for item in rgdp_data["BEAAPI"]["Results"]["Data"]:
    if item["LineNumber"] == "1":
        rgdp_records.append({"Year": int(item["TimePeriod"]), "RGDP": float(item["DataValue"].replace(",", ""))})
df_rgdp = pd.DataFrame(rgdp_records)

# Extract GDP Deflator Data
gdp_deflator_records = []
for item in gdp_deflator_data["BEAAPI"]["Results"]["Data"]:
    if item["LineNumber"] == "1":
        gdp_deflator_records.append({"Year": int(item["TimePeriod"]), "GDP Deflator": float(item["DataValue"].replace(",", ""))})
df_gdp_deflator = pd.DataFrame(gdp_deflator_records)

# Extract Unemployment Data
unemployment_records = []
for item in unemployment_data["BEAAPI"]["Results"]["Data"]:
    if item["LineNumber"] == "1":
        unemployment_records.append({"Year": int(item["TimePeriod"]), "Unemployment Rate": float(item["DataValue"].replace(",", ""))})
df_unemployment = pd.DataFrame(unemployment_records)

# Merge datasets on Year
df_correlation = df_rgdp.merge(df_gdp_deflator, on="Year", how="inner").merge(df_unemployment, on="Year", how="inner")

# Compute the correlation matrix
correlation_matrix = df_correlation[["RGDP", "GDP Deflator", "Unemployment Rate"]].corr()

# Plot the heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap: RGDP, Inflation (GDP Deflator), and Unemployment")
plt.show()




KeyError: 'Results'

In [4]:
print(unemployment_data)


{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID', 'ParameterValue': 'AF24D2AB-0062-43E0-A879-E47F0BAACE62'}, {'ParameterName': 'METHOD', 'ParameterValue': 'GETDATA'}, {'ParameterName': 'DATASETNAME', 'ParameterValue': 'NIPA'}, {'ParameterName': 'TABLENAME', 'ParameterValue': 'T600001'}, {'ParameterName': 'FREQUENCY', 'ParameterValue': 'A'}, {'ParameterName': 'YEAR', 'ParameterValue': 'ALL'}, {'ParameterName': 'RESULTFORMAT', 'ParameterValue': 'JSON'}, {'ParameterName': 'ShowMillions', 'ParameterValue': 'N'}]}, 'Error': {'APIErrorDescription': 'Error retrieving NIPA data.', 'APIErrorCode': '201', 'ErrorDetail': {'Description': 'Invalid TableName was requested.'}}}}
